# NLP  Morderno em Python

Este notebook foi feito para aprendizagem de NLP seguindo o  passo a passo do Patrick Harrison, Director of AI Engineering @ S&P Global, disponível [aqui](https://github.com/skipgram/modern-nlp-in-python-2019/blob/master/notebooks/Modern_NLP_in_Python.ipynb). Este notebook é provido como está e não representa nenhuma garantia. Além disso todo credito deve ser dado ao Patrick Harrison e não a mim.
Você pode ver as explicações dele da versão de 2016 no [Youtube](https://www.youtube.com/watch?v=6zm9NC9uRkk)

# 1. Tópicos:

Este tutorial apresenta um pipeline de processamento de ciência de dados e linguagem natural de ponta a ponta, começando com dados brutos e executando a preparação, modelagem, visualização e análise dos dados. Vamos abordar os seguintes pontos:

* Um tour pelo conjunto de dados
* Introdução ao processamento de texto com spaCy
* Modelagem automática de frases
* Modelagem de tópicos com LDA
* Visualizando modelos de tópicos com pyLDAvis
* Modelos de vetores de palavras com word2vec
* Visualizando o word2vec com t-SNE
* Categorização de texto (classificação) com o modelo textcat do spaCy
* Vetores de palavras contextuais com spaCy Pytorch Transformers
* ... e podemos até aprender uma coisa ou duas sobre Python ao longo do caminho.

Vamos começar!

# 2. O conjunto de dados Yelp

O conjunto de dados Yelp é um conjunto de dados publicado pelo Yelp, serviço de análise de negócios, para fins acadêmicos e educacionais. É um ótimo conjunto de dados do Yelp para demonstrações de aprendizado de máquina e processamento de linguagem natural, porque é grande (mas não tão grande que você precisa de seu próprio data center para processá-lo), bem conectado e qualquer pessoa pode se relacionar com ele - é sobre comida, afinal!

Nota: Se você deseja executar este notebook de forma interativa em sua máquina local, precisará fazer o download de sua própria cópia do conjunto de dados do Yelp. Se você estiver revisando uma cópia estática do notebook online, poderá pular esta etapa. Veja como obter o conjunto de dados:

* Visite a página do conjunto de dados do Yelp [aqui](https://www.yelp.com/dataset)
* Clique em "Download Dataset"
* Leia, concorde e respeite os termos de uso do Yelp!
* O conjunto de dados é baixado como um arquivo .tar; desarquivar
* Coloque os arquivos do conjunto de dados não compactados (business.json etc.) em um diretório chamado yelp_dataset
* Coloque o diretório yelp_dataset no diretório de dados na pasta do projeto Modern NLP in Python

É isso aí! Você está pronto para rodar.

A iteração atual do conjunto de dados do Yelp (a partir desta demonstração) consiste nos seguintes dados:


* 209 mil negócios
* 8 milhões de avaliações de usuários
* 4,5GB comprimidos

<font color='red'>ATENÇÃO: se fizer o download dos dados compactados terá que fazer o unzip algumas vezes, se atente para como você vai organizar as pastas pois será importante na hora de passar o caminho para carregar os aquivos</font>

> atualizado 08/04/2020  

Ao focar apenas nos restaurantes, existem aproximadamente 59 mil restaurantes com aproximadamente 4,2 milhões de comentários de usuários escritos sobre eles.

Os dados são fornecidos em um punhado de arquivos no formato .json. Usaremos os seguintes arquivos para nossa demonstração:

* **business.json** - os registros para empresas individuais

* **review.json** - os registros para comentários que os usuários escreveram sobre empresas

Os arquivos são arquivos de texto (UTF-8) com um objeto json por linha, cada um correspondendo a um registro de dados individual. Vamos dar uma olhada em alguns exemplos.

In [1]:
import os

#Lembre-se que o caminho onde você salvou o dataset pode ser diferente do meu, 
#então ajuste o comando abaixo de acordo coma  estrutura das suas pastas
data_directory = os.path.join('yelp_dataset','_yelp_dataset')

businesses_filepath = os.path.join(data_directory,'yelp_academic_dataset_business.json')

with open(businesses_filepath) as f:
    first_business_record = f.readline() 

print(first_business_record)

{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}



Os registros comerciais consistem em pares de *chave-valor* que contêm informações sobre os negócios específicos. Alguns atributos em que estaremos interessados nesta demonstração incluem:

* **business_id** - identificador exclusivo para empresas
* **categories** - uma lista delimitada por vírgula que contém os valores de categoria relevantes das empresas

O atributo de categorias é de interesse especial. Esta demonstração será focada em restaurantes, indicados pela presença da tag Restaurantes na lista de categorias. Além disso, a lista de categorias pode conter informações mais detalhadas sobre restaurantes, como o tipo de comida que eles servem.

Os registros de revisão são armazenados de maneira semelhante - pares de chave e valor contendo informações sobre as revisões.

In [2]:
review_json_filepath = os.path.join(data_directory, 'yelp_academic_dataset_review.json')

with open(review_json_filepath) as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

Alguns atributos a serem observados nos registros de revisão:

* **business_id** - indica de que empresa trata a revisão
* **texto** - o texto que o usuário escreveu

O atributo text será nosso foco hoje!

## 2.1 Preparando os dados

json é um formato de arquivo útil para troca de dados, mas normalmente não é o mais utilizável para qualquer tipo de trabalho de modelagem. Vamos fazer um pouco mais de preparação de dados para obter nossos dados em um formato mais utilizável. Nosso próximo bloco de código fará o seguinte:

1. Leia em cada registro comercial e converta-o em um dicionário Python
1. Filtre os registros comerciais que não são sobre restaurantes (ou seja, não estão na categoria "Restaurante")
1. Crie um conjunto completo de IDs comerciais para restaurantes, que usaremos na próxima etapa

In [3]:
import json

restaurant_ids = set()

# abra o arquivo de negócios
with open(businesses_filepath,  encoding='utf-8') as f:
    
    # faça a ineracao sobre cada linha
    for business_json in f:
        
        # converta cada registro em json  para um dict do Python
        business = json.loads(business_json)
        
        # Se não existir o atributo 'categories' passe para o proximo,
        if not business.get('categories'):
            continue
        
        # Se o negócio não for restaurante passe para o proximo,
        if 'Restaurants' not in business['categories']:
            continue
            
        # adicione a id do restaurante ao conjunto restaurant_ids
        restaurant_ids.add(business['business_id'])

# transforme restaurant_ids em frozenset, uma vez que não temos quer altera-lo mais
restaurant_ids = frozenset(restaurant_ids)

# o numero de restaurantes
print(f'{len(restaurant_ids):,} restaurantes no dataset.')

63,944 restaurantes no dataset.


Em seguida, criaremos um novo arquivo que contenha apenas o texto das avaliações sobre restaurantes, com uma avaliação por linha no arquivo.

In [4]:
scratch_directory = os.path.join('scratch')

# cria o diretorio scratch directory se ele não existir
try:
    os.mkdir(scratch_directory)
except FileExistsError:
    pass

review_txt_filepath = os.path.join(scratch_directory, 'review_text_all.txt')

In [5]:
# Isso leva um bom tempo executando. Se quiser fazer isso da primeira vez coloque execute = True

execute = False

if execute:
    
    review_count = 0

    # cria e abre o novo arquivo no modo escrita
    with open(review_txt_filepath, 'w', encoding='utf-8') as review_txt_file:

        # abre o arquivos de avaliação json
        with open(review_json_filepath,  encoding='utf-8') as review_json_file:

            # faz o loop através das avaliação e converte em dicionário
            for review_json in review_json_file:
                review = json.loads(review_json, encoding='utf-8')

                # se a avaliação não for sobre restaurante passe para o próximo
                if review['business_id'] not in restaurant_ids:
                    continue

                # escreve a avaliação do restaurante no arquivo como uma linha
                # adiciona uma new line além do existente na avaliação
                review_txt_file.write(review['text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print(f'Um total de {review_count:,} avaliação de restaurantes escritas arquivo txt.')
    
else:
    
    # Apenas conta o número de avaliação que existe no arquivo txt
    with open(review_txt_filepath, encoding='utf-8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(f'Um total de {review_count:,} avaliação de restaurantes escritas arquivo txt.')

Um total de 5,058,161 avaliação de restaurantes escritas arquivo txt.


# 3. spaCy —  NLP nível Industrial em Python

spaCy é uma biblioteca de processamento de linguagem natural -  **Natural Language Processing (NLP)** de nível industrial para Python. O objetivo da spaCy é retirar os recentes avanços no processamento de linguagem natural dos documentos de pesquisa e colocá-los nas mãos dos usuários para criar software de produção.

O spaCy lida com muitas tarefas comumente associadas à construção de um pipeline de processamento de linguagem natural de ponta a ponta:

* Tokenização
* Normalização de texto, como minúsculas, lematização e análise de forma de token
* Marcação de parte da fala
* Análise de dependência sintática
* Detecção de limite de sentença
* Reconhecimento e anotação de entidade nomeada

Seguindo a tradição "pilhas inclusas" do Python, o spaCy contém dados e modelos internos que você pode usar imediatamente para processar texto de idioma inglês de uso geral:

* Grande vocabulário em inglês, incluindo listas de palavras irrelevantes
* "Probabilidades" de Tokens
* Vetores de palavras

<font color='green'> Nota do Tradutor: O spaCy também tem o Português, entretanto os modelos não são tão avançados quanto para o Inglês. </font>

spaCy é escrito em Cython otimizado, o que significa que é rápido. De acordo com algumas fontes independentes, é o analisador sintático mais rápido disponível em qualquer idioma. As principais partes do pipeline de análise spaCy são escritas em C puro, permitindo multithreading eficiente (ou seja, spaCy pode liberar o GIL).

<font color='red'> Atenção!: Se você ainda não tiver instalado execute as linhas abaixo, caso contrário apenas ignore </font>. Para transforma a célula em código novamente, selecione a celula depois click no menu Cell> Cell Type> Code, ou selecione a celula e aperte Y.

## 3.1 Inciando o Spacy

In [6]:
import spacy
from spacy import displacy
import pandas as pd
import itertools as it

nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_md') No tutorial do Harrison era esse mas parece que foi alterado no spaCy

Vamos pegar uma amostra de avaliação para brincar.

In [7]:
review_num = 754601
# atenção este numero pode trazer uma avaliação diferente da apresenta aqui, 
# aconteceu essa diferença entre o tutorial do Harrison e este

with open(review_txt_filepath, encoding='utf-8') as f:
    sample_review = list(it.islice(f, review_num, review_num+1))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

Stopped in here for an after work drink and some ribs. First impression when I walk in is the place is dirty. Patio is niceand modern but somebody should be cleaning the tables better.
I had the Rib and Wing combo which was ok. I cannot complain but I cannot tell you that I was impressed either. It was supposed to be the special of the day but it just looked like they cut the price and the portion size. Fries were good though!
The key is at this restaurant is the service, or should i say the lack of service. Both bartender and waitress never smile, are not very attentive and do not seem to care about being there. This is probably a result of the management/ownership lack of attentiveness. I saw the owner and he looked more like an overseer then someone who was pushing his team to succeed. 
Stay away, until the ownership understands what it means to be in the hospitality business.



------
Passe a avaliação para o spaCy, e se prepare para esperar

In [8]:
%%time
parsed_review = nlp(sample_review)

Wall time: 66.8 ms


---
... uma fração de segundo. Vamos ver o que ele estava fazendo durante este tempo

In [9]:
print(parsed_review)

Stopped in here for an after work drink and some ribs. First impression when I walk in is the place is dirty. Patio is niceand modern but somebody should be cleaning the tables better.
I had the Rib and Wing combo which was ok. I cannot complain but I cannot tell you that I was impressed either. It was supposed to be the special of the day but it just looked like they cut the price and the portion size. Fries were good though!
The key is at this restaurant is the service, or should i say the lack of service. Both bartender and waitress never smile, are not very attentive and do not seem to care about being there. This is probably a result of the management/ownership lack of attentiveness. I saw the owner and he looked more like an overseer then someone who was pushing his team to succeed. 
Stay away, until the ownership understands what it means to be in the hospitality business.



------
Parece a mesma coisa! O que aconteceu sob o capô?

E quanto à detecção e segmentação de sentenças?

In [10]:
for num, sentence in enumerate(parsed_review.sents):
    print(f'Sentence {num + 1}:')
    print(sentence)
    print('')

Sentence 1:
Stopped in here for an after work drink and some ribs.

Sentence 2:
First impression when I walk in is the place is dirty.

Sentence 3:
Patio is niceand modern but somebody should be cleaning the tables better.


Sentence 4:
I had the Rib and Wing combo which was ok.

Sentence 5:
I cannot complain

Sentence 6:
but I cannot tell you that I was impressed either.

Sentence 7:
It was supposed to be the special of the day

Sentence 8:
but it just looked like they cut the price and the portion size.

Sentence 9:
Fries were good though!


Sentence 10:
The key is at this restaurant is the service, or should i say the lack of service.

Sentence 11:
Both bartender and waitress never smile, are not very attentive and do not seem to care about being there.

Sentence 12:
This is probably a result of the management/ownership lack of attentiveness.

Sentence 13:
I saw the owner and he looked more like an overseer then someone who was pushing his team to succeed. 


Sentence 14:
Stay away,

----
E a normalização de texto, como lematização e análise de forma de token?

In [11]:
token_text = [token.orth_ for token in parsed_review]
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(
    zip(token_text, token_lemma, token_shape),
    columns=['token_text', 'token_lemma', 'token_shape']
    )

,token_text,token_lemma,token_shape
0,Stopped,stop,Xxxxx
1,in,in,xx
2,here,here,xxxx
3,for,for,xxx
4,an,an,xx
...,...,...,...
184,the,the,xxx
185,hospitality,hospitality,xxxx
186,business,business,xxxx
187,.,.,.


----
E sobre *part of speech tagging?*

In [12]:
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(
    zip(token_text, token_pos),
    columns=['token_text', 'part_of_speech']
    )

,token_text,part_of_speech
0,Stopped,VERB
1,in,ADV
2,here,ADV
3,for,ADP
4,an,DET
...,...,...
184,the,DET
185,hospitality,NOUN
186,business,NOUN
187,.,PUNCT


---
E a detecção de entidades?

In [13]:
displacy.render(parsed_review, style="ent",jupyter=True)

In [14]:
for num, entity in enumerate(parsed_review.ents):
    print(f'Entity {num + 1}:', entity, '-', entity.label_)
    print('')

Entity 1: First - ORDINAL

Entity 2: Rib - LOC

Entity 3: the day - DATE



<font color='green'> Nota do Tradutor: Parece que nem tudo é perfeito no mundo. O spaCy reconheceu "Rib" como uma localização sendo que é um prato "costela"</font>

---
E a análise de entidade no nível de token?

In [15]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(
    zip(token_text, token_entity_type, token_entity_iob),
    columns=['token_text', 'entity_type', 'inside_outside_begin']
    )

,token_text,entity_type,inside_outside_begin
0,Stopped,,O
1,in,,O
2,here,,O
3,for,,O
4,an,,O
...,...,...,...
184,the,,O
185,hospitality,,O
186,business,,O
187,.,,O


---
E uma variedade de outros atributos no nível de token, como a frequência relativa de tokens, e se um token corresponde ou não a alguma dessas categorias?

* stopword
* pontuação
* espaço em branco
* representa um número
* se o token está ou não incluído no vocabulário padrão do spaCy?

In [16]:
token_attributes = [
        (
        token.orth_,
        token.prob,
        token.is_stop,
        token.is_punct,
        token.is_space,
        token.like_num,
        token.is_oov
        ) for token in parsed_review
    ]

df = pd.DataFrame(
    token_attributes,
    columns=[
        'text',
        'log_probability',
        'stop?',
        'punctuation?',
        'whitespace?',
        'number?',
        'out of vocab.?'
        ]
    )

df.loc[:, 'stop?':'out of vocab.?'] = (
    df
    .loc[:, 'stop?':'out of vocab.?']
    .applymap(lambda x: 'Yes' if x else '')
    )
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Stopped,-20.0,,,,,Yes
1,in,-20.0,Yes,,,,Yes
2,here,-20.0,Yes,,,,Yes
3,for,-20.0,Yes,,,,Yes
4,an,-20.0,Yes,,,,Yes
...,...,...,...,...,...,...,...
184,the,-20.0,Yes,,,,Yes
185,hospitality,-20.0,,,,,Yes
186,business,-20.0,,,,,Yes
187,.,-20.0,,Yes,,,Yes


Se o texto que você processar é Inglês de uso geral (i.e., não é de um domínio específico, como literatura médica), O spaCy está pronto para o uso imediato.

Penso que acabará por se tornar uma parte essencial do ecossistema de ciência de dados Python - fará pela computação em linguagem natural o que outras grandes bibliotecas fizeram para a computação numérica.

# Phrase Modeling 

A *Phrase Modeling* é outra abordagem para aprender combinações de tokens que juntos representam conceitos significativos de várias palavras. Podemos desenvolver *phrase model* repetindo as palavras em nossas análises e procurando por palavras que co-ocorram (ou seja, aparecem uma após a outra) juntas com muito mais frequência do que você esperaria que elas sugissem por acaso. A fórmula para nossos *phrase models* que usaremos para determinar se dois token $A$ and $B$ constitute a phrase is:



$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:

* $count(A)$ é o número de vezes que o token $A$ aparece no corpus
* $count(B)$ é o número de vezes que o token $B$ aparece no corpus
* $count(A\ B)$ é o número de vezes que os tokens $A\ B$ aparecem no corpus na ordem exata
* $N$ é o tamanho total do vocabulário do corpus
* $count_{min}$  é um parâmetro definido pelo usuário para asegurar que a frase aceita ocorre ao menos uma quantidade mínima de vezes
* $threshold$ é um parâmetro definido pelo usuário para controlar a intensidade que deve ter a relação entre os dois tokes para que seja aceita como um *phrase model*


Uma vez que nosso modelo de frase foi treinado em nosso corpus, podemos aplicá-lo ao novo texto. Quando nosso modelo encontra dois tokens no novo texto que é identificado como uma frase, ele mescla os dois em um único novo token.

A *phrase modeling* é superficialmente semelhante à detecção de entidades nomeadas, na medida em que você esperaria que as entidades nomeadas se tornassem frases no modelo (para que new york se tornasse new_york). Mas você também espera que expressões com várias palavras que representem conceitos comuns, mas que não sejam entidades nomeadas especificamente (como happy hour), também se tornem frases no modelo.

Nós nos voltamos para a indispensável biblioteca [gensim](https://radimrehurek.com/gensim/index.html) para nos ajudar na modelagem de frases - a classe [Phrases](https://radimrehurek.com/gensim/models/phrases.html) em particular.

<font color='red'> Atenção!: Se você ainda não tiver instalado execute as linhas abaixo, caso contrário apenas ignore </font>. Para transforma a célula em código novamente, selecione a celula depois click no menu Cell> Cell Type> Code, ou selecione a celula e aperte Y.

In [17]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence

unable to import 'smart_open.gcs', disabling that module


Enquanto realizamos a modelagem de frases, faremos algumas transformações de dados iterativas ao mesmo tempo. Nosso roteiro para a preparação de dados inclui:

* Segmente o texto de avaliações completas em frases e normalize o texto
* Modelagem de frase de primeira ordem $ \rightarrow $ aplica o modelo de frase de primeira ordem para transformar sentenças
* Modelagem de frase de segunda ordem $ \rightarrow $ aplica o modelo de frase de segunda ordem para transformar sentenças
* Aplicar normalização de texto e modelo de frase de segunda ordem ao texto de revisões completas
* Usaremos esses dados transformados como entrada para algumas abordagens de modelagem de nível superior nas seções a seguir.

Primeiro, vamos definir algumas funções auxiliares que usaremos para normalização de texto.

In [18]:
def punct_space(token):
    """
    funcao de auxilio para eliminar tokens
    que sao so pontuacao e espaco em branco
    """
    
    return token.is_punct or token.is_space

def pronoun_lemmatize(token):
    """
    funcao de auxilio para preservar pronomes e tansforma em minuscula enquanto faz lematizacao
    """
    
    if token.lemma_ == '-PRON-':
        return token.lower_
    
    else:
        return token.lemma_.lower()

def line_review(filename):
    """
    funcao geradora para ler avaliacoes do arquivo 
    e desmarque as quebras de linha originais no texto
    """
    
    with open(filename,encoding='utf-8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

Em seguida, usaremos o spaCy para:

* Fazer iterações sobre as avaliações no arquivo review_txt_all.txt que criamos antes
* Segmentar as revisões em frases individuais
* Remover pontuação e excesso de espaço em branco
* Lematização do texto

... e faça isso com o benefício do multiprocessamento, graças à função nlp.pipe() do spaCy. Escreveremos esses dados novamente em um novo arquivo (sentence_lemmatized_all), com uma sentença normalizada por linha. Durante o processo, também pré-processaremos o texto das avaliações completas, não segmentadas por sentenças, da mesma maneira e salvaremos em um arquivo chamado review_lemmatized_all.

Usaremos todos esses dados posteriormente para aprendizagem dos modelos.

In [19]:
review_lemmatized_filepath = os.path.join(scratch_directory, 'review_lemmatized_all.txt')
sentence_lemmatized_filepath = os.path.join(scratch_directory, 'sentence_lemmatized_all.txt')

⚠️ **<font color='red'>Atenção: se você deseja executar novamente o pré-processamento de texto, a célula seguinte demorou cerca de 18 horas para executar o sentenciamento e lematização de todo o texto de revisão de restaurante no conjunto de dados do Yelp.</font>**

In [20]:
# Isso vai demorar um bom tempo- coloque execute = True
# se você quiser executar estes dados por você mesmo

execute = False

if execute:

    with open(review_lemmatized_filepath, 'w',encoding='utf-8') as review_file:
        with open(sentence_lemmatized_filepath, 'w',encoding='utf-8') as sentence_file:
            
            pipe = nlp.pipe(
                line_review(review_txt_filepath),
                batch_size=10000,
                n_threads=8
                )
            
            for parsed_review in pipe:
                
                # Lematização dos textos das avaliações, removendo pontuação e espaços em branco
                lemmatized_review = ' '.join([
                    pronoun_lemmatize(token)
                    for token in parsed_review
                    if not punct_space(token)
                    ])
                
                # salva o texto de cada avaliação lematixada como uma nova linha no arquivo
                review_file.write(lemmatized_review + '\n')
        
                # Faz iteração sobre cada sentença da avaliação
                for sent in parsed_review.sents:
                    
                    # Lematização do texto de cada sentença
                    lemmatized_sentence = ' '.join([
                        pronoun_lemmatize(token)
                        for token in sent
                        if not punct_space(token)
                        ])
                    
                    # salva o texto de cada lemtização como uma linha no arquivo
                    sentence_file.write(lemmatized_sentence + '\n')

Se seus dados estão organizados como o nosso arquivo sentença_lemmatized_all agora é - um grande arquivo de texto com um documento / sentença por linha - a classe LineSentence do gensim fornece um iterador conveniente para trabalhar com outros componentes do gensim. Ele transmite os documentos / frases do disco, para que você nunca precise reter o corpus inteiro na RAM de uma só vez. Isso permite que você dimensione seu pipeline de modelagem para corpora potencialmente muito grande.

In [21]:
sentences_unigrams = LineSentence(sentence_lemmatized_filepath)

Vamos dar uma olhada em algumas frases de exemplo em nosso novo arquivo transformado.

In [23]:
for sentence_unigrams in it.islice(sentences_unigrams, 80, 100):
    print(' '.join(sentence_unigrams))
    print('')

we also order one sushi roll because we have to try at least one

and we love it

definitely will be come back to try more thing

place be really small and there be only one server but we come when it be really slow and the service be pretty fast

all in all very happy we get to try this place out

10 p.m. on a super bowl sunday

and they be already close

weak no wonder the hard rock be die off

a close friend be in town and so instead of take him to a more well establish joint we decide to try the newly open choolah

we be not disappointed

i be a bit of an amateur chef myself and consider my palate to be fairly sophisticated when it come to all kind of south asian cuisine

this be not authentic indian food we do not have rice and salad bowl but it be good wholesome high quality indian food

i order a bowl of rice yellow daal and the koftas meatball

the daal be perfect the way daal should be cook

the koftas be a little bland for my taste but otherwise not bad

soft and the right te

----
Em seguida, aprenderemos um modelo de frase que vinculará palavras individuais a frases de duas palavras. Esperamos que palavras que juntas representem um conceito específico, como "pin ball", sejam vinculadas para formar um novo token único: "pin_ball".

In [24]:
bigram_model_filepath = os.path.join(scratch_directory, 'bigram_phrase_model')

⚠️ **<font color='red'>Atenção: se você deseja executar o processamento de texto, a célula seguinte demorou cerca de 12 minutos  para executar.</font>**

In [25]:
# isso leva um bom tempo executando - set execute = True
# se você quiser executar por você mesmo.

execute = False

if execute:

    bigram_phrases = Phrases(sentences_unigrams)
    
    # Transforma as frases em objetos "Phraser",
    # o que é otimizado para acelerar a memória
    bigram_phrases = Phraser(bigram_phrases)
    bigram_phrases.save(bigram_model_filepath)

In [26]:
# carrega do disco o modelo finalizado
bigram_phrases = Phraser.load(bigram_model_filepath)

Agora que temos um modelo de frase treinado para pares de palavras, vamos aplicá-lo aos dados das frases de revisão e explorar os resultados.

In [27]:
sentences_bigrams_filepath = os.path.join(scratch_directory, 'sentence_bigram_phrases_all.txt')

⚠️ **<font color='red'>Atenção: se você deseja executar o processamento de texto, a célula seguinte demorou cerca de 17 minutos  para executar.</font>**

In [29]:
# isso leva um bom tempo executando - set execute = True
# se você quiser executar por você mesmo.

execute = True

if execute:

    with open(sentences_bigrams_filepath, 'w',encoding='utf-8') as f:
        
        for sentence_unigrams in sentences_unigrams:
            
            sentence_bigrams = ' '.join(bigram_phrases[sentence_unigrams])
            
            f.write(sentence_bigrams + '\n')

In [30]:
sentences_bigrams = LineSentence(sentences_bigrams_filepath)

In [31]:
for sentence_bigrams in it.islice(sentences_bigrams, 80, 100):
    print(' '.join(sentence_bigrams))
    print('')

we also order one sushi roll because we have to try at least one

and we love it

definitely will be come back to try more thing

place be really small and there be only one server but we come when it be really slow and the service be pretty fast

all in all very happy we get to try this place out

10 p.m. on a super bowl sunday

and they be already close

weak no wonder the hard rock be die off

a close friend be in town and so instead of take him to a more well establish joint we decide to try the newly_open choolah

we be not disappointed

i be a bit of an amateur chef myself and consider my palate to be fairly sophisticated when it come to all kind of south asian cuisine

this be not authentic indian food we do not have rice and salad bowl but it be good wholesome high quality indian food

i order a bowl of rice yellow_daal and the koftas meatball

the daal be perfect the way daal should be cook

the koftas be a little bland for my taste but otherwise not bad

soft and the right te

Parece que a frase modelagem funcionou! Agora vemos frases de duas palavras, como "pin_ball" e "saturday_morning", vinculadas no texto como um único token. Em seguida, treinaremos um modelo de frase de segunda ordem. Aplicaremos o modelo de frase de segunda ordem sobre os dados já transformados, para que combinações incompletas de palavras como "ms_pac man" sejam totalmente associadas a "ms_pac_man".

In [32]:
trigram_model_filepath = os.path.join(scratch_directory, 'trigram_phrase_model')

In [34]:
# isso leva um bom tempo executando - set execute = True
# se você quiser executar por você mesmo.

execute = True

if execute:

    trigram_phrases = Phrases(sentences_bigrams)
    
    # Transforma as frases em objetos "Phraser",
    # o que é otimizado para acelerar a memória
    trigram_phrases = Phraser(trigram_phrases)
    trigram_phrases.save(trigram_model_filepath)

In [35]:
# carrega do disco o modelo finalizado
trigram_phrases = Phraser.load(trigram_model_filepath)

Aplicaremos nosso modelo de frase de segunda ordem treinado em nossas frases transformadas de primeira ordem, escreveremos os resultados em um novo arquivo e exploraremos algumas das frases transformadas de segunda ordem.

In [36]:
sentences_trigrams_filepath = os.path.join(scratch_directory, 'sentence_trigram_phrases_all.txt')

In [ ]:
# isso leva um bom tempo executando - set execute = True
# se você quiser executar por você mesmo.

execute = True

if execute:

    with open(sentences_trigrams_filepath, 'w',encoding='utf-8') as f:
        
        for sentence_bigrams in sentences_bigrams:
            
            sentence_trigrams = ' '.join(trigram_phrases[sentence_bigrams])
            
            f.write(sentence_trigrams + '\n')

In [ ]:
sentences_trigrams = LineSentence(sentences_trigrams_filepath)

In [ ]:
for sentence_trigrams in it.islice(sentences_trigrams, 60, 70):
    print(' '.join(sentence_trigrams))
    print('')

---
Parece que o modelo de frase de segunda ordem foi bem-sucedido. Agora estamos vendo frases de três palavras, como "pin_ball_machine" e "ms_pac_man".

A etapa final do nosso processo de preparação de texto volta ao texto completo das revisões. Vamos executar o texto completo das revisões por meio de um pipeline que aplica nossa normalização de texto e modelos de frase.

Além disso, removeremos as palavras-chave neste momento. Palavras de interrupção são palavras muito comuns, como a, the, e assim por diante, que desempenham funções funcionais na linguagem natural, mas geralmente não contribuem para o significado geral do texto. A filtragem de palavras irrelevantes é um procedimento comum que permite que as técnicas de modelagem de PNL de nível superior se concentrem nas palavras que carregam mais peso semântico.

Por fim, escreveremos o texto transformado em um novo arquivo, com uma revisão por linha.

In [ ]:
review_trigrams_filepath = os.path.join(scratch_directory, 'review_trigrams_all.txt')

⚠️ **<font color='red'>Atenção: se você deseja executar o processamento de texto, a célula seguinte demorou cerca de 30 minutos  para executar.</font>**

In [ ]:
# isso leva um bom tempo executando - set execute = True
# se você quiser executar por você mesmo.

execute = False

if execute:
    
    reviews_lemmatized = LineSentence(review_lemmatized_filepath)

    with open(review_trigrams_filepath, 'w') as f:
        
        for review_unigrams in reviews_lemmatized:
                        
            # apply the first-order and second-order phrase models
            review_bigrams = bigram_phrases[review_unigrams]
            review_trigrams = trigram_phrases[review_bigrams]

            # remove any remaining stopwords
            review_trigrams = [
                term
                for term in review_trigrams
                if term not in nlp.Defaults.stop_words
                ]

            # write the transformed review as a line in the new file
            review_trigrams = ' '.join(review_trigrams)
            f.write(review_trigrams + '\n')